In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["figure.dpi"] = 100
import plotly.graph_objects as go
from scipy.optimize import fsolve
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

In [26]:
a = 0
b = 2
T = 1

k = lambda u, x, t : u
q = lambda u, x, t : 1

f = lambda u, x, t : 0

mu_a = lambda t : 10*t**2
mu_b = lambda t : 0

u0 = lambda x : 0

tau = 0.05
h = 0.05
M = int(T/tau) + 1
N = int((b-a)/h) + 1
U = np.zeros((M, N))
Xs = np.linspace(a, b, N)
Ts = np.linspace(0, T, M)
A = np.zeros((N,N))
B = np.zeros(N)

last_layer = np.zeros(N)

for i in range(N):
    last_layer[i] = u0(i*h)
    
U[0] = last_layer

for m in range(1, M):
    for i in range(100):
        A[0][0] = 1
        B[0] = mu_a(Ts[m])
        for n in range(1, N-1):
            current_point = np.array([
                last_layer[n],
                Xs[n],
                Ts[m]
            ])
            current_point_plus_one = np.array([
                last_layer[n + 1],
                Xs[n+1],
                Ts[m]
            ])
            current_point_minus_one = np.array([
                last_layer[n - 1],
                Xs[n-1],
                Ts[m]
            ])
            current_q = q(current_point[0], current_point[1], current_point[2]);
            current_f = f(current_point[0], current_point[1], current_point[2]);
            current_plus_half_k = (k(current_point_plus_one[0], current_point_plus_one[1], current_point_plus_one[2]) + k(current_point[0], current_point[1], current_point[2])) / 2
            current_minus_half_k = (k(current_point_minus_one[0], current_point_minus_one[1], current_point_minus_one[2]) + k(current_point[0], current_point[1], current_point[2])) / 2
            factor = tau * current_q / h**2
            previous_coefficient = - factor * current_minus_half_k
            next_coefficient = - factor * current_plus_half_k
            current_coefficient = 1 - previous_coefficient - next_coefficient
            A[n][n - 1] = previous_coefficient
            A[n][n] = current_coefficient
            A[n][n + 1] = next_coefficient
            B[n] = last_layer[n] + tau * current_f
        A[-1][-1] = 1
        B[-1] = mu_b(Ts[m]);
        last_layer = np.linalg.solve(A, B)
    U[m] = last_layer

In [27]:
sh_t, sh_x = U.shape
fig = go.Figure()
fig.add_surface(
    # contours = {
    #     "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
    #     "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
    # },
    x = np.linspace(a, b, sh_x),
    y = np.linspace(0, T, sh_t),
    z = U
    )
# fig.add_scatter3d(name='scat_p', x=np.linspace(a, b, sh_1), y=np.linspace(0, T, sh_0), z=U, mode='markers',
#     marker=dict(size=10, color='black',colorscale='Reds'))
# fig.update_layout(
#         scene = {
#             "xaxis": {"nticks": 20},
#             "zaxis": {"nticks": 4},
#             'camera_eye': {"x": 0, "y": -1, "z": 0.5},
#             "aspectratio": {"x": 1, "y": 1, "z": 0.2}
#         })
fig.update_layout(title='surface', autosize=False,
                  width=500, height=500,
                  margin=dict(l=5, r=50, b=5, t=55))
fig.show()